In [25]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

In [26]:
X_train = pd.read_csv('../Datasets/Performance_X_train.csv')
X_test = pd.read_csv('../Datasets/Performance_X_test.csv')
y_train = pd.read_csv('../Datasets/Performance_y_train.csv')

In [27]:
X_train

,id,age,gender,height,weight,body_fat,diastolic,systolic,grip_force,sit_bend_forward,sit_ups,broad_jump
0,920,40,M,173.8,73.2,12.0,80.0,148,46.9,15.0,50.0,250.0
1,5998,35,M,176.2,73.7,15.7,64.0,119,47.6,18.0,46.0,226.0
2,11457,57,M,169.2,73.5,21.7,77.0,128,49.5,14.8,39.0,206.0
3,2898,45,M,170.3,76.7,17.0,86.0,154,50.8,19.5,43.0,214.0
4,3141,28,M,171.3,68.7,11.2,99.0,145,42.7,23.2,71.0,269.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10708,11701,63,F,151.6,58.0,37.2,78.0,137,19.7,21.4,0.0,84.0
10709,12327,56,F,169.0,53.1,17.8,74.0,119,22.2,4.8,32.0,147.0
10710,9592,21,M,168.4,66.7,22.9,70.0,119,27.5,-6.4,36.0,213.0
10711,5221,37,M,174.6,83.0,28.3,99.0,142,39.2,7.9,33.0,177.0


In [28]:
X_test

,id,age,gender,height,weight,body_fat,diastolic,systolic,grip_force,sit_bend_forward,sit_ups,broad_jump
0,11,42,M,169.2,65.4,19.3,63.0,110.0,43.5,16.0,68.0,211
1,22,59,F,155.9,62.7,30.2,76.0,143.0,36.8,29.1,25.0,122
2,30,50,F,159.8,57.1,24.4,63.0,103.0,30.8,24.4,30.0,143
3,49,28,M,180.1,82.1,15.0,83.0,147.0,52.6,18.8,55.0,247
4,88,41,F,158.1,51.0,21.7,86.0,127.0,23.1,28.6,46.0,165
...,...,...,...,...,...,...,...,...,...,...,...,...
2675,13277,55,M,168.4,71.3,27.8,89.0,146.0,37.8,19.1,32.0,189
2676,13312,33,M,173.1,67.6,18.1,83.0,124.0,46.4,9.4,52.0,209
2677,13328,24,M,170.9,68.3,16.4,90.0,147.0,34.4,4.3,33.0,173
2678,13358,56,M,173.0,52.6,12.9,92.0,141.0,32.6,-13.2,31.0,198


In [29]:
y_train

,id,class
0,920,A
1,5998,A
2,11457,A
3,2898,A
4,3141,A
...,...,...
10708,11701,D
10709,12327,D
10710,9592,D
10711,5221,D


In [30]:
ID = X_test['id'].copy()

X_train = X_train.drop(columns=['id'])
X_test = X_test.drop(columns=['id'])
y_train = y_train.drop(columns=['id'])

In [32]:
X_train_cat = X_train.select_dtypes('object').copy()
X_test_cat = X_test.select_dtypes('object').copy()

ohe = OneHotEncoder(sparse=False)
ohe.fit(X_train_cat)

X_train_ohe = ohe.transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

X_train_num = X_train.select_dtypes(exclude='object').copy()
X_test_num = X_test.select_dtypes(exclude='object').copy()

scaler = MinMaxScaler()
scaler.fit(X_train_num)

X_train_sca = scaler.transform(X_train_num)
X_test_sca = scaler.transform(X_test_num)

X_TRAIN = np.concatenate([X_train_ohe, X_train_sca], axis=1)
X_TEST = np.concatenate([X_test_ohe, X_test_sca], axis=1)

y_TRAIN = y_train['class'].map({'A':0, 'B':1, 'C':2, 'D':3})

print(X_TRAIN.shape, X_TEST.shape, y_TRAIN.shape)

(10713, 12) (2680, 12) (10713,)


In [34]:
xtrain, xtest, ytrain, ytest = train_test_split(X_TRAIN, y_TRAIN, test_size=0.25, stratify=y_TRAIN, random_state=2022)

print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(8034, 12) (2679, 12) (8034,) (2679,)


In [43]:
def make_models(xtrain, xtest, ytrain, ytest):
    model2 = DecisionTreeClassifier(random_state=2022).fit(xtrain, ytrain)
    print('model2', get_score(model2, xtrain, xtest, ytrain, ytest))
    
    for d in range(3,8):
        model2 = DecisionTreeClassifier(max_depth=d, random_state=2022).fit(xtrain, ytrain)
        print('model2', d, get_score(model2, xtrain, xtest, ytrain, ytest))
    
    model3 = RandomForestClassifier(random_state=2022).fit(xtrain, ytrain)
    print('model3', get_score(model3, xtrain, xtest, ytrain, ytest))
    
    for d in range(3,8):
        model3 = RandomForestClassifier(500, max_depth=d, random_state=2022).fit(xtrain, ytrain)
        print('model3', get_score(model3, xtrain, xtest, ytrain, ytest))
        
def get_score(model, xtrain, xtest, ytrain, ytest):
    A = model.score(xtrain, ytrain)
    ypred = model.predict_proba(xtest)
    B = roc_auc_score(ytest, ypred, multi_class='ovr')
    
    return f'{A:.4} {B:.4}'

In [44]:
make_models(xtrain, xtest, ytrain, ytest)

model2 1.0 0.7474
model2 3 0.5479 0.7751
model2 4 0.5732 0.8043
model2 5 0.6121 0.8178
model2 6 0.6234 0.8379
model2 7 0.6649 0.8499
model3 1.0 0.9141
model3 0.5896 0.827
model3 0.616 0.8485
model3 0.6535 0.8651
model3 0.6965 0.8774
model3 0.7407 0.8887


In [45]:
final_model = RandomForestClassifier(500, max_depth=8, random_state=2022).fit(xtrain, ytrain)
print('final_model', get_score(final_model, xtrain, xtest, ytrain, ytest))

final_model 0.785 0.8971


In [48]:
y_pred = final_model.predict(X_TEST)

obj = {
    'ID':ID,
    'class':y_pred
}
result = pd.DataFrame(obj)
result['class'] = result['class'].map({0:'A',1:'B',2:'C',3:'D'})
result.to_csv('./result.csv', index=False)